In [1]:
#install deepeval and huggingface dependencies
!pip install transformers
!pip install -U deepeval


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# log into deepeval
# do it in terminal
# deepeval login

In [ ]:
# log into huggingface
# huggingface-cli login

In [29]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.benchmarks import HellaSwag
from deepeval.benchmarks.tasks import HellaSwagTask
import re

In [23]:
# Define benchmark with specific tasks and shots
benchmark = HellaSwag(
    tasks=[HellaSwagTask.TRIMMING_BRANCHES_OR_HEDGES, HellaSwagTask.BATON_TWIRLING],
    n_shots=5
)

In [34]:
class Mistral7B(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        sections = [section.strip() for section in prompt.split("\n\n") if section.strip()]

        # Take the last section, including "Answer:" for context
        prompt = sections[-2]
        
        model = self.load_model()

        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        
        ans = self.tokenizer.batch_decode(generated_ids)[0]

        match = re.search(r"Answer:\s*([A-D])", ans)

        if match:
            answer = match.group(1)
        else:
            answer = 'N/A'

        return answer

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    # This is optional.
    def batch_generate(self, promtps: [str]) -> [str]:
        model = self.load_model()
        device = "cuda" # the device to load the model onto

        new_p = []
        for p in promtps:
            sections = [section.strip() for section in prompt.split("\n\n") if section.strip()]
            new_p.append(sections[-2])

        model_inputs = self.tokenizer(
            new_p,
            padding=True,    # Ensure equal-length inputs
            truncation=True, # Truncate inputs that exceed max_length
            max_length=512,
            return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        decoded_responses = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        res = []
        for ans in decoded_responses:
            match = re.search(r"Answer:\s*([A-D])", ans)
    
            if match:
                res.append(match.group(1))
            else:
                res.append('N/A')
        return res

    def get_model_name(self):
        return "Mistral 7B"

In [25]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
mistral_7b = Mistral7B(model=model, tokenizer=tokenizer)

In [37]:
results = benchmark.evaluate(model=mistral_7b, batch_size=10)
print("Task-specific Scores: ", benchmark.task_scores)
print("Detailed Predictions: ", benchmark.predictions)
print(benchmark.overall_score)

Processing Trimming branches or hedges: 100%|██████████| 4/4 [00:17<00:00,  4.41s/it]


HellaSwag Task Accuracy (task=Trimming branches or hedges): 1.0


Processing Baton twirling: 100%|██████████| 14/14 [00:44<00:00,  3.20s/it]

HellaSwag Task Accuracy (task=Baton twirling): 0.35714285714285715
Overall HellaSwag Accuracy: 0.5
Task-specific Scores:                            Task     Score
0  Trimming branches or hedges  1.000000
1               Baton twirling  0.357143
Detailed Predictions:                             Task  \
0   Trimming branches or hedges   
1   Trimming branches or hedges   
2   Trimming branches or hedges   
3   Trimming branches or hedges   
4                Baton twirling   
5                Baton twirling   
6                Baton twirling   
7                Baton twirling   
8                Baton twirling   
9                Baton twirling   
10               Baton twirling   
11               Baton twirling   
12               Baton twirling   
13               Baton twirling   
14               Baton twirling   
15               Baton twirling   
16               Baton twirling   
17               Baton twirling   

                                                Input Prediction  